![Alt text](sprocket_central_logo.png)

For context, Sprocket Central Pty Ltd is a long-standing KPMG client whom specialises in high-quality bikes and accessible cycling accessories to riders. Their marketing team is looking to boost business by analysing their existing customer dataset to determine customer trends and behaviour. 

Using the existing 3 datasets (Customer demographic, customer address and transactions) as a labelled dataset, please recommend which of these 1000 new customers should be targeted to drive the most value for the organisation. 

Prepare a detailed approach for completing the analysis including activities – i.e. understanding the data distributions, feature engineering, data transformations, modelling, results interpretation and reporting.Please advise what steps you would take.


In [13]:
# Modules
import pandas as pd

In [18]:
# Load the Excel file

workbook = pd.ExcelFile('kmpg-dpocket-data.xlsx')



In [19]:
# Get the sheet names
sheet_names = workbook.sheet_names
print('Sheet names:', sheet_names)

Sheet names: ['Title Sheet', 'Transactions', 'NewCustomerList', 'CustomerDemographic', 'CustomerAddress']


In [27]:
# Transactions

transactions = workbook.parse(sheet_names[1])
transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


## Summarize transactions

There are 2000 transactions in the transaction table
From the summary, there are some comumns that contains missing data.

In [28]:
transactions.describe()

,transaction_id,product_id,customer_id,online_order,list_price,standard_cost,product_first_sold_date
count,20000.000000,20000.00000,20000.000000,19640.000000,20000.000000,19803.000000,19803.000000
mean,10000.500000,45.36465,1738.246050,0.500458,1107.829449,556.046951,38199.776549
std,5773.647028,30.75359,1011.951046,0.500013,582.825242,405.955660,2875.201110
min,1.000000,0.00000,1.000000,0.000000,12.010000,7.210000,33259.000000
25%,5000.750000,18.00000,857.750000,0.000000,575.270000,215.140000,35667.000000
50%,10000.500000,44.00000,1736.000000,1.000000,1163.890000,507.580000,38216.000000
75%,15000.250000,72.00000,2613.000000,1.000000,1635.300000,795.100000,40672.000000
max,20000.000000,100.00000,5034.000000,1.000000,2091.470000,1759.850000,42710.000000


It seems there is a problem here, 197 transactions has a product id of 0, no product line, class, size and brand.
What could be the cause of this?
I would reccommened rows that has this problem are dropped.

In [29]:
print(transactions[transactions['product_class'].isna()].head())

     transaction_id  product_id  customer_id transaction_date  online_order  \
136             137           0          431       2017-09-23           0.0   
159             160           0         3300       2017-08-27           0.0   
366             367           0         1614       2017-03-10           0.0   
406             407           0         2559       2017-06-14           1.0   
676             677           0         2609       2017-07-02           0.0   

    order_status brand product_line product_class product_size  list_price  \
136     Approved   NaN          NaN           NaN          NaN     1942.61   
159     Approved   NaN          NaN           NaN          NaN     1656.86   
366     Approved   NaN          NaN           NaN          NaN      850.89   
406     Approved   NaN          NaN           NaN          NaN      710.59   
676     Approved   NaN          NaN           NaN          NaN     1972.01   

     standard_cost  product_first_sold_date  
136       

Clean transactions,  dropped all rows that contains missing value.

In [30]:
transactions_clean = transactions.dropna() 
transactions_clean.describe()

,transaction_id,product_id,customer_id,online_order,list_price,standard_cost,product_first_sold_date
count,19445.000000,19445.000000,19445.000000,19445.000000,19445.000000,19445.000000,19445.000000
mean,9989.257393,45.797737,1739.467267,0.500849,1107.337193,555.585260,38201.758653
std,5779.669087,30.571996,1011.889153,0.500012,582.662400,405.596541,2878.067854
min,1.000000,0.000000,1.000000,0.000000,12.010000,7.210000,33259.000000
25%,4976.000000,18.000000,857.000000,0.000000,575.270000,215.140000,35667.000000
50%,9985.000000,45.000000,1741.000000,1.000000,1163.890000,507.580000,38216.000000
75%,14997.000000,72.000000,2615.000000,1.000000,1635.300000,795.100000,40672.000000
max,20000.000000,100.000000,5034.000000,1.000000,2091.470000,1759.850000,42710.000000


New customer list


In [33]:
customer_list = workbook.parse(sheet_names[2])
customer_list = customer_list.loc[:, ~customer_list.columns.str.startswith('Unnamed')]
customer_list.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125


In [35]:
customer_list.shape

(1000, 18)

There are 1000 new customers

## Customer Demographic

In [36]:
customer_demo = workbook.parse(sheet_names[3])
customer_demo.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


We have 4000 customers

In [37]:
customer_demo.shape

(4000, 13)

Customer Address
It seems the address of customer id 3 is missing.

In [38]:
customer_address = workbook.parse(sheet_names[4])
customer_address.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [40]:
customer_address.shape

(3999, 6)

## Next line of action

joining the tables,

-  a left join of customer to transactions
- identify customers that are part of the new 1000, some type of join will be required.
- Create a new column profit (list price - standard cost)
- Group by customers 
- Calculate the customers that generated most profit, sort in descending order, and select the highest 
- Visualization
- Prepare dashboard

## Current Contributors
- [@johndata91](https://www.github.com/johndata91)
- [@Kvngbee563412](https://www.github.com/Kvngbee563412)
- [@Oyindamolajames](https://www.github.com/Oyindamolajames)
- [@Favourewoh](https://www.github.com/Favourewoh)

if you want to contribute, let me know so I can add you to the list of contributors.

### For those that are part of the contributors 
I have pushed commits to the main branch, you can make pull request to access my changes on your local repository, and make your contributions.